# Notebook 2: Exploratory_Data_Analysis_Drug
## Introdução
Neste notebook, vamos carregar os dados da tabela `Drug` que foram salvos em formato Parquet no notebook anterior e realizar uma análise exploratória dos dados.

## Passos

1. **Configuração Inicial**
    - Importar bibliotecas necessárias.

2. **Carregar Dados do Parquet**
    - Carregar os dados do arquivo Parquet `drug_raw`.

3. **Mostrar o Schema e as Primeiras Linhas dos Dados**
    - Verificar o schema dos dados carregados.
    - Exibir as primeiras linhas do DataFrame.

4. **Análise Exploratória dos Dados**
    - Contar o número de linhas no DataFrame.
    - Realizar estatísticas para variaveis numericas e nominais.
    - Verificar valores nulos.
    - Analisar coluna por coluna e testar possíveis alterações 

5. **Conclusão**
    - Resumir as descobertas iniciais e preparar os dados para análises futuras.

In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, count, when, initcap, lit, concat_ws, array, array_distinct, regexp_extract, coalesce, lower, isnull, regexp_replace, concat, substring




In [0]:
# Carregar dados do Parquet
df_drug = spark.read.parquet('dbfs:/FileStore/FAERS-grupo-4/drug_raw')

In [0]:
# Mostrar o schema dos dados carregados
df_drug.printSchema()

#####Analisar a tabela drug

In [0]:
# Exibir DataFrame
df_drug.display()


##Metadados

**PRIMARYID**
- Contem o identificador do relatório FAERS, está bem classificado como 'long', pois é um nº, contudo não pode ser nullable. Todos os registos têm de ter um nº de relatório associado.

**CASEID**
- Contem o identificador do episódio FAERS, está bem classificado como 'integer', pois é um nº, contudo não pode ser nullable. Todos os registos têm de ter um nº de episódio associado.

**DRUG_SEQ**
- Número único para identificar um medicamento para um caso específico. Tipo de dados: Integer

**ROLE_COD**
- Código que indica o papel relatado do medicamento no evento adverso. Tipo de dado: string

    Valores Possíveis:
      
        PS: Primary Suspect Drug
        SS: Secondary Suspect Drug
        C: Concomitante
        I: Interagindo

**DRUGNAME**
- Nome do produto do medicamento. Tipo de dado: string

**PROD_AI**
- Ingrediente ativo do produto, quando disponível.Tipo de dado: string 

**VAL_VBM**
- Código para a fonte do nome do medicamento. Tipo de Dado: Integer

    Valores Possíveis:
        1 - Nome comercial validado
        2 - Nome exato usado

**ROUTE**
- Via de administração do medicamento.Tipo de Dado: String

**DOSE_VBM**
- Texto exato para dose, frequência e via de administração, conforme registrado no relatório.Tipo de Dado: String

**CUM_DOSE_CHR** 
- Dose cumulativa até a primeira reação. Tipo de Dado: Double

**CUM_DOSE_UNIT**
- Unidade da dose cumulativa até a primeira reação. Tipo de Dado: String

**DECHAL**
- Código de dechallenge, indicando se a reação cessou quando a terapia com o medicamento foi interrompida. Tipo de Dado: String

    Valores Possíveis:

        Y: Dechallenge positivo
        N: Dechallenge negativo
        U: Desconhecido
        D: Não se aplica
      
**RECHAL**
- Código de rechallenge, indicando se a reação recorreu quando a terapia com o medicamento foi reiniciada. Tipo de Dado: String

    Valores Possíveis:

        Y: Rechallenge positivo
        N: Rechallenge negativo
        U: Desconhecido
        D: Não se aplica


**LOT_NUM**
- Número do lote do medicamento conforme relatado. Tipo de Dado: String

**EXP_DT**
- Data de validade do medicamento no formato YYYYMMDD. Tipo de Dado: Integer

**NDA_NUM**
- Número do NDA (New Drug Application). Tipo de Dado: Long

**DOSE_AMT**
- Quantidade do medicamento relatada. Tipo de Dado: Double

**DOSE_UNIT**
- Unidade da dose do medicamento.Tipo de Dado: String

**DOSE_FORM**
- Forma da dose do medicamento. Tipo de Dado: String

**DOSE_FREQ**
- Código para a frequência da dose do medicamento. Tipo de Dado: String

    Valores Possíveis:
    
       1X: Uma vez
       BID: Duas vezes ao dia
       TID: Três vezes ao dia
       QD: Diariamente
       PRN: Conforme necessário
       ...


In [0]:
df_drug.count()

A tabela tem 9480689 registos.
A maioria das variaveis sao nominais ou categoricas

## Estatísticas descritivas

In [0]:
# Selecionar colunas numéricas
columns_describe = ["cum_dose_chr", "exp_dt" , "drug_seq", "dose_amt"]

# Estatísticas descritivas das colunas
df_drug.select(columns_describe).describe().display()


Algumas análises com pouca relevância, é necessário aprofundar...  

## Contagem da percentagem de valores nulos inicial

In [0]:
total_rows = df_drug.count()

null_percentages = {}

# Iterar sobre as colunas e calcular porcentagens de valores nulos
for col_name in df_drug.columns:
    null_count = df_drug.where(col(col_name).isNull()).count()
    null_percentage = (null_count / total_rows) * 100
    null_percentages[col_name] = null_percentage

# Exibir porcentagens de valores nulos por coluna
for col_name, percentage in null_percentages.items():
    print(f"Coluna '{col_name}' possui {percentage:.2f}% de valores nulos.")


### Tabela DRUG

- Ao analisar percebo que as colunas primaryid, caseid, drug_seq, role_cod, drugname, val_vbm são essenciais para preenchimento desta tabela, com percentagem de null de 0,00%

- Já colunas como a cum_dose_chr, cum_dose_unit e ainda exp_dt tem taxas de null proximas de 100%, nao permitindo inferenciar sobre as mesmas, no enanto, opta-se por manter estas variaveis e colunas uma vez que, podem ser vantajosas para análises especificas.

- Sendo as colunas primaryid, caseid, drug_seq, role_cod, drugname e val_vbm essenciais para esta tabela, nao foram realizadas alterações , no entanto, para compreensao dos dados, relizaram-se algumas analises.




## ROLE_COD

In [0]:
df_drug.groupBy('ROLE_COD').count().orderBy('count', ascending=False).display()

Na tabela Drugs, o campo Role_Cod deve estar sempre preenchido. Existem 2 valores a null, sendo esta uma variavel categorica, opta-se por substituir os 2 valores a null por a moda desta variavel (SS). 

In [0]:
df_drug = df_drug.withColumn(
    "Role_Cod",
    when(col("Role_Cod").isNull(), "SS").otherwise(col("Role_Cod"))
)


In [0]:
#Confirmar se já não existem valores null
df_drug.groupBy('ROLE_COD').count().orderBy('count', ascending=False).display()

As variaveis apresentam-se em siglas. Para uma melhor perceção e interpretação dos resultados opta-se por substituir as variavies pelo que as mesmas representam.

In [0]:
df_drug.withColumn(
    "ROLE_COD",
    when(col("ROLE_COD") == "PS", "Primary Suspect Drug")
    .when(col("ROLE_COD") == "SS", "Secondary Suspect Drug")
    .when(col("ROLE_COD") == "C", "Concomitant")
    .when(col("ROLE_COD") == "I", "Interaction")
    .otherwise("Unknown")
).display()

## drugname VS prod_ai VS val_vbm

O sistema FAERS de fato possui um dicionário formal, mas não está disponível publicamente. A principal fonte do dicionário da FDA é o Structured Product Labeling (SPL), no entanto, o link que daria  nomes de medicamentos do SPL não se encontrava disponíveis durante esta sessão. 
Também utilizam outras referências que não estão disponíveis publicamente (por exemplo, o Dicionário de Medicamentos da Organização Mundial da Saúde (WHO-DD), RxNorm do NIH, etc.). 
Nos dados pode-se apresentar uma consistência nos dados de medicamentos e fabricantes, pois agora são menos dependentes de registos textuais para passarem a teer vocabulário controlado do dicionário de dados. 








In [0]:
# Mostrar os registros filtrados para compreender como são apresentados os dados que tem valores null no prod_ai
df_drug.filter(col('prod_ai').isNull()).display()


Cmo já analisado anteriormente 100% dos dados apresentam drugname, no entanto, alguns destes nomes não sao validados pelos dicionarios, tal como podemos verificar na colona val_vbm, que indica se o nome é validado ou se encontra-se em verbatim, ou seja, tal como descrito. 

2,18% dos dados nao apresenta prod_ai, no entanto, este dado considera-se imprescidivel para efetivamente qual a substancia causadora da reaçao.

In [0]:
# Filtrar registros onde 'val_vbm' é igual a 2
df_val_vbm_2 = df_drug.filter(col('val_vbm') == 2)

# Contar o número de registros filtrados
count_val_vbm_2 = df_val_vbm_2.count()

print(f"Número de registros onde 'val_vbm' é 2: {count_val_vbm_2}")

In [0]:

# Contar o número total de registros
total_count = df_drug.count()

# Filtrar os registros onde val_vbm é igual a 2
count_val_vbm_2 = df_drug.filter(col('val_vbm') == 2).count()

# Calcular a porcentagem
percentage_val_vbm_2 = (count_val_vbm_2 / total_count) * 100

# Exibir o resultado
print(f"A percentagem de registos com val_vbm igual a 2 é: {percentage_val_vbm_2:.2f}%")

In [0]:
from pyspark.sql.functions import col, isnull

# Filtrar registros onde 'val_vbm' é igual a 2 e 'prod_ai' é NULL
df_val_vbm_2_null_prod_ai = df_drug.filter((col('val_vbm') == 2) & (isnull(col('prod_ai'))))

# Contar o número de registros filtrados
count_val_vbm_2_null_prod_ai = df_val_vbm_2_null_prod_ai.count()

print(f"Número de registros onde 'val_vbm' é 2 e 'prod_ai' é NULL: {count_val_vbm_2_null_prod_ai}")


In [0]:
from pyspark.sql.functions import col, isnull

# Filtrar registros onde 'val_vbm' é igual a 2 e 'prod_ai' é NULL
df_val_vbm_2_null_prod_ai = df_drug.filter((col('val_vbm') == 2) & (isnull(col('prod_ai'))))

# Contar o número de registros filtrados
count_val_vbm_2_null_prod_ai = df_val_vbm_2_null_prod_ai.count()

print(f"Número de registros onde 'val_vbm' é 2 e 'prod_ai' é NULL: {count_val_vbm_2_null_prod_ai}")


In [0]:

# Filtrar registros onde 'prod_ai' é nulo e 'val_vbm' é igual a 1
df_prod_ai_null_val_vbm_1 = df_drug.filter((col('prod_ai').isNull()) & (col('val_vbm') == 1))

# Opcional: Contar o número de registros filtrados
count_prod_ai_null_val_vbm_1 = df_prod_ai_null_val_vbm_1.count()
print(f"Número de registros onde 'prod_ai' é nulo e 'val_vbm' é 1: {count_prod_ai_null_val_vbm_1}")

In [0]:
from pyspark.sql.functions import col, isnull

# Contar o número total de registros
total_count = df_drug.count()

# Filtrar os registros onde val_vbm é igual a 2 ou prod_ai é NULL
count_val_vbm_2_or_null_prod_ai = df_drug.filter((col('val_vbm') == 2) | (isnull(col('prod_ai')))).count()

# Calcular a porcentagem
percentage_val_vbm_2_or_null_prod_ai = (count_val_vbm_2_or_null_prod_ai / total_count) * 100

# Exibir o resultado
print(f"A percentagem de registos com val_vbm igual a 2 ou prod_ai é NULL é: {percentage_val_vbm_2_or_null_prod_ai:.2f}%")


Compreendendo a baixa percentagem de registos de apresentam null na seu prod_ai (2.18%) e a sua alta associaçao com nome do medicamento nao validado pelo dicionario e ainda a dificuldade de aceder publicamente aos dicionarios de medicamentos da FAERS, opta-se por eliminar os registos que apresentem null no prod_ai e val_vbm=2, para, posteriormente, obter analises mais fidedignas e menos enviesadas.

Com isto, tambem para uma analise mais objetiva da tabela, uma vez que todos os registos apresentaram o nome validado, opta-se por eliminar esta coluna. 

In [0]:

# Filtrar os registros onde prod_ai é null e val_vbm é igual a 2 + eliminar coluna val_vbm
df_filtered = df_drug.filter((col('prod_ai').isNotNull()) & (col('val_vbm') != 2))

df_filtered = df_drug.drop('val_vbm')

df_filtered.display()



##Route

In [0]:
df_drug.groupBy('ROUTE').count().orderBy('count', ascending=False).display()

- Percebendo que existe somente um injetion e um injetable e, neste contexto os dados nao fazem sentido, opta-se por colocar estes 2 valores como null
-Tambem existe 3 descriçoes para a administraçao via intravenous "drip" , "bolus" e "(not otherwise specified)", pelo que se opta por unfiormizar os dados e colocar uma so descriçao para todos os valores. 

In [0]:
df_drug_newroute = df_drug.withColumn(
    "ROUTE",
    when((col("ROUTE") == "Unknown") | (col("ROUTE") == "INJECTION")| (col("ROUTE") == "INJECTABLE")|(col("ROUTE") == "Other"), None)
    .when((col("ROUTE") == "Intravenous drip") | 
          (col("ROUTE") == "Intravenous (not otherwise specified)") | 
          (col("ROUTE") == "Intravenous bolus"), "Intravenous")
    .otherwise(col("ROUTE"))
)

df_drug_newroute.display()

##Dose_vbm
- É uma tabela verbatim text sendo sugerido o formato dose, frequency, and route
- Uma vez que esta coluna tem menos percentagem de null e pode-se obter informaçoes importantes para preencher outras colunas, vamos tentar uniformizar alguns dados desta tabela

In [0]:
distinct_dosevbm = df_drug.select('dose_vbm').distinct()

# Mostrar os valores distintos
distinct_dosevbm.display()

# Opcional: Contar o número de valores distintos
count_distinct_dosevbm = distinct_dosevbm.count()

In [0]:
##Palavras que podem ser implementadas da coluna route 

# Lista de palavras a serem procuradas
words = [
    "Oral", "Intravenous", "Subcutaneous", "Intravenous drip", "Intramuscular", "Topical", "Transplacental", 
    "Respiratory (inhalation)", "Ophthalmic", "Intra-uterine", "Transdermal", "Intracardiac", "Intravenous bolus", 
    "Nasal", "Intrathecal", "Rectal", "Intraperitoneal", "Cutaneous", "Endocervical", "Vaginal", "Intraocular", 
    "Sublingual", "Intra-arterial", "Buccal", "Parenteral", "Intra-articular", "Intracavernous", "Urethral", 
    "Epidural", "Transmammary", "Intravesical", "Subconjunctival", "Intradermal", "Hemodialysis", "Dental", 
    "Intralesional", "Periarticular", "Intrameningeal", "Oropharyngeal", "Auricular (otic)", "Endosinusial", 
    "Intra-amniotic", "Endotracheal", "Intradiscal (intraspinal)", "Intrapleural", "Occlusive dressing technique", 
    "Intratumor", "Intracervical", "Intracisternal", "Intramedullar (bone marrow)", "Intra corpus cavernosum", 
    "Intrapericardial", "Intrathoracic", "Intratracheal", "Iontophoresis", "Intracoronary", "Intrasynovial", 
    "Perineural", "Retrobulbar", "Intracerebral", "Intrahepatic", "Intralymphatic", "Intracorneal", "Extra-amniotic"
]

# Criar as colunas temporárias para cada palavra e combiná-las em uma única coluna 'routenew'
df_drug = df_drug.withColumn(
    'routenew',
    coalesce(*[when(col('dose_vbm').rlike(f"(?i)\\b{word}\\b"), initcap(lit(word))) for word in words])
)

# Criar a coluna 'routenew2' combinando 'routenew' e 'route'
df_drug = df_drug.withColumn(
    'routenew2',
    when(col('route').isNotNull(), col('route')).otherwise(col('routenew'))
)

# Mostrar o DataFrame atualizado
df_drug.select('dose_vbm', 'route', 'routenew', 'routenew2').display()

- Opta-se por não realizar a mesma sequencia para as dosagens dos medicamentos porque podem exitir erros ao associar as doses uma vez que, em modo verbatim, a pessoa pode estar a referenciar a dose_unit, a cum_dose_unit ou a dosagem do frasco ou solução, entres outras coisas, o que se torna mais dificil inferir nestas unidades de medida


In [0]:

# Ajustar a coluna dose_vbm
df_drug.withColumn(
    "dose_vbm",
    # Substituir "unk" em qualquer posição e em qualquer formato (maiúsculas ou minúsculas)
    regexp_replace(col("dose_vbm"), "(?i)unk", "")
).display()

# Remover vírgulas ou pontos no início ou fim dos registros
df_drug = df_drug.withColumn(
    "dose_vbm",
    # Substituir vírgulas ou pontos no início ou fim dos registros
    regexp_replace(col("dose_vbm"), "^[,\\.]+|[,\\.]+$", "")
)


- Por a quantidade de UNK e unkwon presente, opta-se por eliminar estas expressões desta coluna, uma vez que nao trazem relevancia.
- As analises nesta coluna dose_vbm deve ser evitada para a analise porque torna-se dificil opter resultados concretos e objetivos 

As Coluna 'cum_dose_chr' e 'cum_dose_unit'  possuem 97.64% de valores nulos, esta colunas estao aparentemente corretas e não se iram realizar alterações

##Dechal
Pode apresentar os seguintes valores:
 - Y: Dechallenge positivo
 - N: Dechallenge negativo
 - U: Desconhecido
 - D: Não se aplica

Para melhor perceção do tabela opta-se por substiuir os valores pelo seu significado e, considerando campo de desconhecido como irrelevante, opta-se por o substituir por null

In [0]:

df_drug_dechal = df_drug.withColumn(
    "dechal",
    when(col("dechal") == "Y", "Positive")
    .when(col("dechal") == "N", "Negative")
    .when(col("dechal") == "U", None)
    .when(col("dechal") == "D", "Does not apply")
    .otherwise(col("dechal"))
)

df_drug_dechal.select("dechal").distinct().display()

##Rechal
Pode apresentar os seguintes valores:
 - Y: Rechallenge positivo
 - N: Rechallenge negativo
 - U: Desconhecido
 - D: Não se aplica

Para melhor perceção do tabela opta-se por substiuir os valores pelo seu significado e, considerando campo de desconhecido como irrelevante, opta-se por o substituir por null

In [0]:
df_drug_rechal = df_drug.withColumn(
    "rechal",
    when(col("rechal") == "Y", "Positive")
    .when(col("rechal") == "N", "Negative")
    .when(col("rechal") == "U", None)
    .when(col("rechal") == "D", "Does not apply")
    .otherwise(col("rechal"))
)

df_drug_rechal.select("rechal").distinct().display()

#LOT_NUM

In [0]:
df_drug.groupBy('LOT_NUM').count().orderBy('count', ascending=False).display()


- As analisar as expressoes e percetivel que existem valores incorretos ou mal escritos. Vamos uniformizar os dados e, para tal, considera-se imporante eliminar dados se encontrem depois de , ou ; ou | ou / naao sendo esta a representaçao padrao do num_ lote . Opta-se por se deixar os . e - 
- Eliminam-se os espaços
- Tenta-se tambem limpar algumas expressoes indesejadas expressoes_indesejadas como UNK|unknown|NA|Not|Available|ASKED|BUT|reported|time|at|this|asku|N/A , que se encontram presentes em diversas celulas

In [0]:

# Limpar a coluna LOT_NUM
df_drug = df_drug.withColumn(
    "LOT_NUM",
    # Remover tudo após ,;|/
    regexp_replace("LOT_NUM", r"[,;|/].*$", "")
)

#Remover os espaços em branco
df_drug = df_drug.withColumn("LOT_NUM", regexp_replace("LOT_NUM", r"\s+", ""))


# Remover expressões indesejadas 
expressoes_indesejadas = [
    r"(?i)\b(UNK|unknown|NA|Not|Available|ASKED|BUT|reported|time|at|this|asku|N/A)\b"
]

for expressao in expressoes_indesejadas:
    df_drug = df_drug.withColumn("LOT_NUM", regexp_replace("LOT_NUM", expressao, ""))


In [0]:
# Mostrar DataFrame atualizado
df_drug.select("LOT_NUM").distinct().display()

##EXP_DT
- Está em formato YYYYMMDD, opta-se por colocar em formato yyyy-mm-dd para se tornar mais percetivel 
- A coluna 'exp_dt' possui 99.83% de valores nulos pelo que torna-se dificil tira conclusoes a partir desta variavel posteriormente, no entanto, em casos especifos, pode ser util, pelo que opto por mante-la na tabela final 


In [0]:

# Verificar e formatar a coluna EXP_DT
df_drug = df_drug.withColumn(
    "EXP_DT",
    when(
        col("EXP_DT").rlike(r"^\d{8}$"),
        concat(
            substring(col("EXP_DT"), 1, 4), lit("-"),
            substring(col("EXP_DT"), 5, 2), lit("-"),
            substring(col("EXP_DT"), 7, 2)
        )
    ).otherwise(col("EXP_DT"))
)

df_drug.select("EXP_DT").display()

## NDA_NUM
Representa o número da New Drug Application, sendo este um identificador composto por números, nao se realizam alteraçoes nesta coluna

## DOSE_AMT
Representa a quantidade de medicamento por dose. Não se irão realizar alterções nesta coluna. 

##DOSE_UNIT
Unidade da dose do medicamento.


In [0]:
df_drug.groupBy('DOSE_UNIT').count().orderBy('count', ascending=False).display()


##Dose_form

Variavel em verbatim... que pretende indicar a forma da substancia causadora da reaçao

In [0]:
df_drug.groupBy('DOSE_FORM').count().orderBy('count', ascending=False).display()


Sendo vias de administraçao, nao pode haver números, pelo que vamos eliminar os números existentes

In [0]:



df_drug = df_drug.withColumn(
    "DOSE_FORM",
    when(
        col("DOSE_FORM").isin("Unknown", "UNK", "NOT SPECIFIED", "Formulation Unknown", 
                              "Unknown Formulation", "Not Provided", "Not Specified", 
                              "Unknown (other/unspecified)", "***", "No Drug", "Xxx", "Unknown,unknown", " ()"  ),
        None
    ).otherwise(
        initcap(
            regexp_replace(col("DOSE_FORM"), r"\d+", "")
        )
    )
)


vamos mapear os dados considerando os formatos de solucões mais frequentes

In [0]:
# Colocar em minusculas
df_drug = df_drug.withColumn(
    "DOSE_FORM",
    lower(col("DOSE_FORM"))
)


df_drug = df_drug.withColumn(
    "DOSE_FORM",
    when(col("DOSE_FORM").rlike(".*tablets?"), "Tablet")
    .when(col("DOSE_FORM").rlike(".*solutions?"), "Solution")
    .when(col("DOSE_FORM").rlike(".*injections?"), "Injection")
    .when(col("DOSE_FORM").rlike(".*capsules?"), "Capsule")
    .when(col("DOSE_FORM").rlike(".*powders?"), "Powder")
    .when(col("DOSE_FORM").rlike(".*infusions?"), "Infusion")
    .when(col("DOSE_FORM").rlike(".*orals?"), "Oral")
    .when(col("DOSE_FORM").rlike(".*creams?"), "Cream")
    .when(col("DOSE_FORM").rlike(".*gels?"), "Gel")
    .when(col("DOSE_FORM").rlike(".*ointments?"), "Ointment")
    .when(col("DOSE_FORM").rlike(".*suspensions?"), "Suspension")
    .when(col("DOSE_FORM").rlike(".*drops?"), "Drops")
    .when(col("DOSE_FORM").rlike(".*nasal sprays?"), "Nasal Spray")
    .when(col("DOSE_FORM").rlike(".*patches?"), "Transdermal Patch")
    .when(col("DOSE_FORM").rlike(".*dry powders?"), "Dry Powder")
    .when(col("DOSE_FORM").rlike(".*pens?"), "Pen")
    .when(col("DOSE_FORM").rlike(".*aerosols?"), "Aerosol")
    .when(col("DOSE_FORM").rlike(".*drinks?"), "Drink")
    .when(col("DOSE_FORM").rlike(".*pumps?"), "Pump")
    .when(col("DOSE_FORM").rlike(".*injects?"), "Injection")
    .when(col("DOSE_FORM").rlike(".*liquid topicals?"), "Liquid Topical")
    .when(col("DOSE_FORM").rlike(".*rectals?"), "Rectal")
    .when(col("DOSE_FORM").rlike(".*emulsions?"), "Emulsion")
    .when(col("DOSE_FORM").rlike(".*solvents?"), "Solvent")
    .when(col("DOSE_FORM").rlike(".*liquid"), "Liquid")
    .when(col("DOSE_FORM").rlike(".*topicals?"), "Topical")
    .when(col("DOSE_FORM").rlike(".*capsulas?"), "Capsulas")
    .when(col("DOSE_FORM").rlike(".*implantations?"), "Implantation")
    .when(col("DOSE_FORM").rlike(".*«syringes?"), "Injection")
    .when(col("DOSE_FORM").rlike(".*iv"), "Injection")
    .when(col("DOSE_FORM").rlike(".*emulsions?"), "Emulsion")
    .when(col("DOSE_FORM").rlike(".*lotions?"), "Lotion")
    .when(col("DOSE_FORM").rlike(".*vapours?"), "Vapour")
    .when(col("DOSE_FORM").rlike(".*dispersions?"), "Dispersion")
    .when(col("DOSE_FORM").rlike(".*granules?"), "Granule")
    .when(col("DOSE_FORM").rlike(".*sprays?"), "Spray")
    .otherwise(initcap(col("DOSE_FORM")))
)

# Mostrar DataFrame atualizado para verificar as alterações
df_drug.select("DOSE_FORM").distinct().display()



In [0]:
df_drug.groupBy('DOSE_FORM').count().orderBy('count', ascending=False).display()

Considerando que a grande maioria dos dados ja apresenta nomenclatura mais uniforme e objetiva,opta-se por colocaros restantes valores a null para o pipeline

##Dose_freq

- A frequencia com que o medicamento é tomado 

In [0]:
df_drug.groupBy('DOSE_FREQ').count().orderBy('count', ascending=False).display()


Há valores que nao fazem parte do dicionario... vamos adaptar as siglas para o nome pos extenso para ficar percetivel ... vamos ajustar os valores que sao possiveis de corrigir e eliminar os que nao se adaptam e consideramos errados.

In [0]:


# Substituir as siglas pela descrição completa
df_drug = df_drug.withColumn(
    "DOSE_FREQ",
    when(col("DOSE_FREQ") == "1X", "Once or one time")
    .when(col("DOSE_FREQ") == "BID", "Twice a day")
    .when(col("DOSE_FREQ") == "BIW", "Twice a week")
    .when(col("DOSE_FREQ") == "HS", "At bedtime")
    .when(col("DOSE_FREQ") == "PRN", "As needed")
    .when(col("DOSE_FREQ") == "Q12H", "Every 12 hours")
    .when(col("DOSE_FREQ") == "Q2H", "Every 2 hours")
    .when(col("DOSE_FREQ") == "Q3H", "Every 3 hours")
    .when(col("DOSE_FREQ") == "Q3W", "Every 3 weeks")
    .when(col("DOSE_FREQ") == "Q4H", "Every 4 hours")
    .when(col("DOSE_FREQ") == "Q5H", "Every 5 hours")
    .when(col("DOSE_FREQ") == "Q6H", "Every 6 hours")
    .when(col("DOSE_FREQ") == "Q8H", "Every 8 hours")
    .when(col("DOSE_FREQ") == "QD", "Daily")
    .when(col("DOSE_FREQ") == "QH", "Every hour")
    .when(col("DOSE_FREQ") == "QID", "4 times a day")
    .when(col("DOSE_FREQ") == "QM", "Monthly")
    .when(col("DOSE_FREQ") == "QOD", "Every other day")
    .when(col("DOSE_FREQ") == "QOW", "Every other week")
    .when(col("DOSE_FREQ") == "QW", "Every week")
    .when(col("DOSE_FREQ") == "TID", "3 times a day")
    .when(col("DOSE_FREQ") == "TIW", "3 times a week")
    .when(col("DOSE_FREQ") == "UNK", None)  # Substituir "UNK" por null
    .when(col("DOSE_FREQ").like("%/WK"), "Every week")  # Quando termina com /WK
    .when(col("DOSE_FREQ").like("%/MONTH"), "Monthly")  # Quando termina com /MONTH
    .when(col("DOSE_FREQ").like("%/HR"), "Every hour")
    .when(col("DOSE_FREQ").like("%/MIN"), "Every minute")  # Quando termina com /HR
    .when(col("DOSE_FREQ") == "/CYCLE", "Every cicle")
    .when(col("DOSE_FREQ") == "/YR", "Every year")
    .when(col("DOSE_FREQ") == "TRIMESTER", "Every 3 months")
    .when(col("DOSE_FREQ") == "/SEC", "Ever")
    .when(col("DOSE_FREQ") == "999", None)  # Substituir "999" por null
    .when(col("DOSE_FREQ").isin("PC", "ONCE", "Q4-6H", "QAM", "TOTAL", "AC", "UD"), None)  # Substituir "PC", "ONCE", "Q4-6H" por null
    .otherwise(col("DOSE_FREQ"))  # Manter outros valores sem alteração
)


df_drug.groupBy('DOSE_FREQ').count().orderBy('count', ascending=False).display()




Conseguimos adaptar para ter todos os registos uniformes... Transformando /WK em "Every week" , /MONTH em "Monthly" , /HR em "Every hour", 
 /MIN em "Every minute", /CYCLE em "Every cicle", /YR em "Every year", TRIMESTER em "Every 3 months", /SEC em "Ever" e ainda eliminar valores sem sentido transformando-os em nul como 999, TOTAL", "AC", "UD"... 

#Conclusões

###Notas Importantes
- Quando val_vbm = 2 encontra-se em verbatim pelo que nao se consegue ligaçao ao dicionario e torna-se dificil chegar a conclusoes, motivo pelo qual vao ser eliminados os registos os este valor 
- Colunas com alta percetagens de null como cum_ e exp_dt, são dificies de tirar inferencia, no entanto nao são eliminados porque podem ser úteis para conclusoes especificas


#Correções
- Susbtituir nulls da coluna Role_cod pela moda e substituir as siglas pela sua interpretaçao.
- Eliminar registos onde o prod_ai is null ou val_vbm=2 , perdendo cerca de 2,18% dos dados em prol de uma analise mais objetiva e fidedigna 
- Considerando que todos os registos da coluna val_vbm serão = 1, opta-se por eliminar esta coluna. 
- Na coluna Route substuiu-se o Unknow, injection e injectable por null e uniformizar as expressóes intravenous
- Na coluna dose_vbm retirar as expressões UNK e unkwon para melhor perceção
- Utilizar expressoes da coluna dose_vbm para preencher espaço da coluna route que se encontrem a null
- Eliminar tabelas route antiga (com menos informação)
- Na coluna Dechal dar significado aos campos e colocar a null os campos desconhecidos (com a letra U)
- Na coluna Rechal dar significado aos campos e colocar a null os campos desconhecidos (com a letra U)
- No coluna Lot_NUm  deve-se eliminar dados se encontrem depois de , ou ; ou | ou / , Eliminam-se os espaço e limpar algumas expressoes indesejadas expressoes_indesejadas como UNK|unknown|NA|Not|Available|ASKED|BUT|reported|time|at|this|asku|N/A 
- Na coluna LOT_NUM passar a null valores vazios
- Na coluna EXP_DT passar as datas para formato yyyy-mm-dd
- Passar coluna EXP_DT para formato data
- Na coluna Dose_form limpar e uniformizar os dados 
- Na coluna Dose_freq ajusta-se os horarios de acordo com os dicionarios e adaptar os registos possiveis. De seguida, eliminar os desconhecidos ou sem sentido (passando-os a valores null)
- Renomear as colunas 

